In [162]:
import os
import pandas as pd
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [163]:
def get_files(dir_path: list) -> dict:
    files = [file for file in os.listdir(dir_path)]
    file_paths = [dir_path + file for file in files]
    return pd.DataFrame({"file_name": files, "file_path": file_paths})


def read_file(file_path):
    with open(file_path, "r") as file:
        return file.read()


def get_text(db: pd.DataFrame) -> pd.DataFrame:
    db["text"] = db["file_path"].apply(read_file)
    return db


# def get_context(db: pd.DataFrame, overlap: int = 300) -> pd.DataFrame:
#     context = []
#     for row in db.iterrows():
#         i = row["chunk"].metadata["start_index"]
#         if i - overlap < 0 or i + overlap > len(row["text"]):
#             continue

#         context.append(row["text"][i - overlap : i + overlap])

#     db["context"] = context
#     return db


def get_context_for_row(row, chunk_size, overlap):
    start_index = row["chunk"].metadata["start_index"]
    text_length = len(row["text"])

    if start_index - overlap < 0:
        return row["text"][start_index : start_index + chunk_size + overlap]
    if start_index + chunk_size + overlap > text_length:
        return row["text"][start_index - overlap : text_length]

    return row["text"][start_index - overlap : start_index + chunk_size + overlap]


def get_context(db: pd.DataFrame, chunk_size, overlap: int = 100) -> pd.DataFrame:
    # Apply the function to each row of the DataFrame
    db["context"] = db.apply(
        get_context_for_row, chunk_size=chunk_size, overlap=overlap, axis=1
    )
    return db


def get_chunks(db: pd.DataFrame, text_splitter) -> pd.DataFrame:
    db["chunk"] = db["text"].apply(lambda s: text_splitter.create_documents([s]))
    return db.explode("chunk")


def get_embeddings(db: pd.DataFrame, model) -> pd.DataFrame:
    db["embedding"] = db["chunk"].apply(lambda s: model.encode(s.page_content))
    return db


def separate_tables(db):
    return db[["file_name", "file_path", "text"]].drop_duplicates(
        subset="file_name"
    ), db.drop(columns=['file_path', 'text'])


def to_postgres():
    NotImplemented


CHUNK_SIZE = 512

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)
db = get_files(os.getcwd() + "\data\\")
db = get_text(db)
db = get_chunks(db, text_splitter)
db = get_context(db, CHUNK_SIZE)
db = get_embeddings(db, model)
text_db, vector_db = separate_tables(db)
#to_postgres(vector_db)

In [164]:
text_db

,file_name,file_path,text
0,2226.txt,c:\Users\nicho\Documents\legislation_llm\pipel...,To authorize appropriations for fiscal year 2...
1,3076.txt,c:\Users\nicho\Documents\legislation_llm\pipel...,[Congressional Bills 117th Congress]\n[From th...


In [165]:
vector_db

,file_name,chunk,context,embedding
0,2226.txt,page_content='To authorize appropriations for ...,To authorize appropriations for fiscal year 20...,"[-0.04015888, 0.0014534396, 0.03527895, -0.018..."
0,2226.txt,"page_content=""(a) Short Title.--This Act may b...",e \nUnited States of America in Congress assem...,"[-0.03192492, -0.020027928, -0.0030170872, -0...."
0,2226.txt,page_content='Subtitle A--Authorization of App...,DEPARTMENT OF DEFENSE AUTHORIZATIONS\n\n ...,"[-0.069968805, 0.0012165786, 0.009229354, -0.0..."
0,2226.txt,page_content='Sec. 111. Report on Army require...,tions\n\nSec. 101. Authorization of appropriat...,"[-0.065443374, 0.06627812, -0.008602901, -0.04..."
0,2226.txt,page_content='augmentation program of the Army...,on plants.\nSec. 115. Report on acquisition st...,"[-0.092744105, -0.031237988, 0.04267691, -0.00..."
...,...,...,...,...
1,3076.txt,"page_content='Regulatory Commission, but such ...",pending on the effective \n date of this s...,"[-0.035138424, 0.07144515, 0.031928845, -0.011..."
1,3076.txt,page_content='(4) Suits.--This section and the...,en discontinued or modified \n if this sect...,"[-0.03640648, 0.09893074, 0.059135232, -0.0388..."
1,3076.txt,page_content='any document relating to the Ins...,"any other Federal law, \n Executive order,...","[-0.008682428, 0.07932113, 0.126732, -0.013632..."
1,3076.txt,page_content='TITLE III--SEVERABILITY\n\n S...,ect on the date that is 180 days after the dat...,"[-0.060060788, -0.008277961, 0.039201688, -0.0..."


In [183]:
# CONFIG

Your instance connection name is: legistlation-llm:us-east4:legislation-llm-vectordb


In [184]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()


# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME, "pg8000", user=DB_USER, password=DB_PASS, db=DB_NAME
    )
    return conn


# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

In [185]:
pool

Engine(postgresql+pg8000://)

In [187]:
with pool.connect() as db_conn:
    db_conn.execute(sqlalchemy.text("CREATE TABLE * FROM legislation_vector_db_001"))

DatabaseError: (pg8000.exceptions.DatabaseError) {'S': 'ERROR', 'V': 'ERROR', 'C': '42P01', 'M': 'relation "legislation_vector_db_001" does not exist', 'P': '15', 'F': 'parse_relation.c', 'L': '1384', 'R': 'parserOpenTable'}
[SQL: SELECT * FROM legislation_vector_db_001]
(Background on this error at: https://sqlalche.me/e/20/4xp6)

In [186]:
# connect to connection pool
with pool.connect() as db_conn:
    # create ratings table in our sandwiches database
    db_conn.execute(
        sqlalchemy.text(
            "CREATE TABLE IF NOT EXISTS ratings "
            "( id SERIAL NOT NULL, name VARCHAR(255) NOT NULL, "
            "origin VARCHAR(255) NOT NULL, rating FLOAT NOT NULL, "
            "PRIMARY KEY (id));"
        )
    )

    # commit transaction (SQLAlchemy v2.X.X is commit as you go)
    db_conn.commit()

    # insert data into our ratings table
    insert_stmt = sqlalchemy.text(
        "INSERT INTO ratings (name, origin, rating) VALUES (:name, :origin, :rating)",
    )

    # insert entries into table
    db_conn.execute(
        insert_stmt, parameters={"name": "HOTDOG", "origin": "Germany", "rating": 7.5}
    )
    db_conn.execute(
        insert_stmt, parameters={"name": "BÀNH MÌ", "origin": "Vietnam", "rating": 9.1}
    )
    db_conn.execute(
        insert_stmt,
        parameters={"name": "CROQUE MADAME", "origin": "France", "rating": 8.3},
    )

    # commit transactions
    db_conn.commit()

    # query and fetch ratings table
    results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()

    # show results
    for row in results:
        print(row)

(1, 'HOTDOG', 'Germany', 7.5)
(2, 'BÀNH MÌ', 'Vietnam', 9.1)
(3, 'CROQUE MADAME', 'France', 8.3)
